In [1]:
!sudo apt-get update && sudo apt-get install cmake libopenmpi-dev zlib1g-dev

Get:1 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ InRelease [3,626 B]
Get:2 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  InRelease [1,581 B]
Get:3 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  Packages [874 kB]
Get:4 http://security.ubuntu.com/ubuntu jammy-security InRelease [129 kB]
Hit:5 http://archive.ubuntu.com/ubuntu jammy InRelease
Hit:6 https://ppa.launchpadcontent.net/c2d4u.team/c2d4u4.0+/ubuntu jammy InRelease
Hit:7 https://ppa.launchpadcontent.net/deadsnakes/ppa/ubuntu jammy InRelease
Get:8 http://security.ubuntu.com/ubuntu jammy-security/main amd64 Packages [1,857 kB]
Get:9 http://archive.ubuntu.com/ubuntu jammy-updates InRelease [128 kB]
Hit:10 https://ppa.launchpadcontent.net/graphics-drivers/ppa/ubuntu jammy InRelease
Hit:11 https://ppa.launchpadcontent.net/ubuntugis/ppa/ubuntu jammy InRelease
Get:12 http://security.ubuntu.com/ubuntu jammy-security/universe amd64 Packages [1,084 kB]
Get:13 http://

In [2]:
!apt-get update && apt-get install ffmpeg freeglut3-dev xvfb  # For visualization
!pip install "stable-baselines3[extra]>=2.0.0a4"

Hit:1 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ InRelease
Hit:2 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  InRelease
Hit:3 http://security.ubuntu.com/ubuntu jammy-security InRelease
Hit:4 http://archive.ubuntu.com/ubuntu jammy InRelease
Hit:5 http://archive.ubuntu.com/ubuntu jammy-updates InRelease
Hit:6 http://archive.ubuntu.com/ubuntu jammy-backports InRelease
Hit:7 https://ppa.launchpadcontent.net/c2d4u.team/c2d4u4.0+/ubuntu jammy InRelease
Hit:8 https://ppa.launchpadcontent.net/deadsnakes/ppa/ubuntu jammy InRelease
Hit:9 https://ppa.launchpadcontent.net/graphics-drivers/ppa/ubuntu jammy InRelease
Hit:10 https://ppa.launchpadcontent.net/ubuntugis/ppa/ubuntu jammy InRelease
Reading package lists... Done
Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
ffmpeg is already the newest version (7:4.4.2-0ubuntu0.22.04.1).
The following additional packages will be installed:
  freeglut3 libegl-d

In [3]:
import gym
#import gym_examples
import numpy as np

print(f"{gym.__version__=}")

gym.__version__='0.25.2'


In [4]:
from stable_baselines3.ppo import MlpPolicy
from stable_baselines3 import PPO, DQN
from stable_baselines3.common.evaluation import evaluate_policy

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [5]:
import gymnasium as gym
from gymnasium import spaces
import numpy as np

class NQueensEnv(gym.Env):
    def __init__(self, n=8):
        super(NQueensEnv, self).__init__()
        self.n = n  # Size of the board
        self.action_space = spaces.Discrete(n * n)  # Action space
        self.observation_space = spaces.MultiDiscrete([n] * n)  # Observation space
        self.current_step = 0
        self.max_steps = 500
        self.reset()

    def reset(self, **kwargs):
        self.state = np.random.choice(self.n, size=self.n, replace=False)  # Random initial state
        self.current_step = 0
        return self.state, {}

    def step(self, action):
        self.current_step= self.current_step +1
        row = action % self.n
        col = action // self.n
        self.state[row] = col
        reward = self.calculate_reward()
        if reward == 0:
          done = True
          truncated = False
        elif  self.current_step >= self.max_steps:
          truncated = True
          done = False
        else:
          done = False
          truncated = False
        return self.state, reward, done, truncated, {}

    def calculate_reward(self):
        # Initialize violations count
        violations = 0
        # Create sets to keep track of conflicts
        row_set = set()
        diag1_set = set()
        diag2_set = set()

        # Iterate through each queen
        for i in range(self.n):
            # Calculate positions on diagonals
            diag1 = self.state[i] + i
            diag2 = self.state[i] - i

            # Check for conflicts
            if self.state[i] in row_set or diag1 in diag1_set or diag2 in diag2_set:
                violations += 1

            # Update sets
            row_set.add(self.state[i])
            diag1_set.add(diag1)
            diag2_set.add(diag2)

        # Reward is negative number of violations
        return -violations

    def is_terminal_state(self):
        return self.calculate_reward() == 0  # Terminal state reached when there are no violations

    def render(self, mode='human'):
        board = [['_' for _ in range(self.n)] for _ in range(self.n)]
        for i, row in enumerate(board):
            row[self.state[i]] = 'Q'
            print(' '.join(row))

# Example of creating and using the environment
if __name__ == "__main__":
    env = NQueensEnv()
    state, _ = env.reset()
    print("Initial state:")
    env.render()
    # Example of taking a step
    action = env.action_space.sample()
    #print(action)
    state, reward, done, truncated, info = env.step(action)
   # print("\nState after one step:")
    print(state)
    env.render()
    print(f"Reward: {reward}, Done: {done}")


Initial state:
_ _ Q _ _ _ _ _
_ _ _ _ _ _ Q _
_ _ _ _ _ _ _ Q
_ Q _ _ _ _ _ _
_ _ _ _ Q _ _ _
Q _ _ _ _ _ _ _
_ _ _ _ _ Q _ _
_ _ _ Q _ _ _ _
[2 6 7 1 4 0 1 3]
_ _ Q _ _ _ _ _
_ _ _ _ _ _ Q _
_ _ _ _ _ _ _ Q
_ Q _ _ _ _ _ _
_ _ _ _ Q _ _ _
Q _ _ _ _ _ _ _
_ Q _ _ _ _ _ _
_ _ _ Q _ _ _ _
Reward: -2, Done: False


/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [6]:
# import gym
# import stable_baselines3
# from stable_baselines3 import PPO
# from stable_baselines3.common.callbacks import BaseCallback
# from stable_baselines3.common.buffers import RolloutBuffer
# import stable_baselines3.common.off_policy_algorithm
# import torch
# import numpy as np
# from stable_baselines3.common.callbacks import BaseCallback

# class CustomCallback(BaseCallback):
#     def __init__(self, model, verbose=0):
#         super(CustomCallback, self).__init__(verbose)
#         self.model = model

#     def _on_step(self) -> bool:

#         return True


# def collect_rollouts(env, callback, rollout_buffer, n_rollout_steps):
#     obs, info = env.reset()
#     for _ in range(n_rollout_steps):
#         # Convert observation tuple to NumPy array
#         action, _states = model.predict(obs)
#         new_obs, rewards, done, truncated, info = env.step(action)
#         rollout_buffer.add(new_obs,action,rewards,False,torch.zeros(1,1),torch.zeros(1, 1))
#         obs = new_obs
#         callback.on_step()
#         if done:
#             obs = env.reset()
#     return rollout_buffer.size() >= n_rollout_steps



In [ ]:
# # model = PPO('MlpPolicy', env, verbose=1).learn(10000)
# model = PPO('MlpPolicy', env, verbose=1).learn(1000000)

# # Initialize callback
# callback = CustomCallback(model)

# # Initialize rollout buffer
# buffer_size = 300
# obs_space = env.observation_space
# action_space = env.action_space
# rollout_buffer = RolloutBuffer(buffer_size, obs_space, action_space, gamma=0.99)
# #print(rollout_buffer.size())
# obs, info = env.reset()
# model._last_obs = obs
# # Number of rollout steps to collect
# n_rollout_steps = 300

# # Collect rollouts
# success = collect_rollouts(env,callback, rollout_buffer, n_rollout_steps)

# if success:
#     print("Collected the desired number of rollout steps.")
# else:
#     print("Rollout was terminated prematurely by the callback.")

In [16]:
import gym
import torch
import numpy as np
from stable_baselines3 import PPO
from stable_baselines3.common.callbacks import BaseCallback
import pickle

class CustomCallback(BaseCallback):
    def __init__(self, verbose=0):
        super(CustomCallback, self).__init__(verbose)
        self.states = []
        self.actions = []
        self.rewards = []

    def _on_step(self) -> bool:
        # Retrieve local variables

        obs = self.locals['obs_tensor']
        actions = self.locals['actions']
        rewards = self.locals['rewards']

        # Store the collected data
        self.states.append(obs)
        self.actions.append(actions)
        self.rewards.append(rewards)

        return True

    def on_training_end(self):
        # Optionally print or save the collected data
        print(f"Collected states: {self.states}")
        print(f"Collected actions: {self.actions}")
        print(f"Collected rewards: {self.rewards}")
        liste = []
        dataset = {}
        if 'rewards' not in dataset:
            dataset['rewards'] = []
        if 'actions' not in dataset:
            dataset['actions'] = []
        if 'states' not in dataset:
            dataset['states'] = []
        dataset['states'] = self.states
        dataset['actions'] = self.actions
        dataset['rewards'] = self.rewards
        liste.append(dataset)
        with open('data.pkl', 'wb') as file:
          pickle.dump(liste, file)

        # # For demonstration, we will save the collected data to files
        # np.save('collected_states.npy', np.array(self.states))
        # np.save('collected_actions.npy', np.array(self.actions))
        # np.save('collected_rewards.npy', np.array(self.rewards))


In [17]:

# Create the callback
callback = CustomCallback()
model = PPO('MlpPolicy', env, verbose=1)
# Train the model and collect data using the callback
model.learn(total_timesteps=1000000, callback=callback)

Streaming output truncated to the last 5000 lines.
|    value_loss           | 0.797       |
-----------------------------------------
-----------------------------------------
| rollout/                |             |
|    ep_len_mean          | 7.22        |
|    ep_rew_mean          | -14.9       |
| time/                   |             |
|    fps                  | 395         |
|    iterations           | 252         |
|    time_elapsed         | 1306        |
|    total_timesteps      | 516096      |
| train/                  |             |
|    approx_kl            | 0.021142315 |
|    clip_fraction        | 0.113       |
|    clip_range           | 0.2         |
|    entropy_loss         | -0.263      |
|    explained_variance   | 0.957       |
|    learning_rate        | 0.0003      |
|    loss                 | 0.375       |
|    n_updates            | 2510        |
|    policy_gradient_loss | -0.0309     |
|    value_loss           | 0.832       |
-------------------------

IOPub data rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_data_rate_limit`.

Current values:
NotebookApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
NotebookApp.rate_limit_window=3.0 (secs)



In [19]:
import pickle

# Load the data from the .pkl file
with open('data.pkl', 'rb') as file:
    loaded_data = pickle.load(file)


In [24]:
for i in range(1001472):
  loaded_data[0]['states'][i] = loaded_data[0]['states'][i][0]
  loaded_data[0]['actions'][i] = loaded_data[0]['actions'][i][0]
  loaded_data[0]['rewards'][i] = loaded_data[0]['rewards'][i][0]

In [27]:
loaded_data[0]['states'][1001471]

tensor([4, 6, 1, 3, 7, 0, 2, 1], device='cuda:0')

In [ ]:
with open('Nqueens.pkl', 'wb') as file:
  pickle.dump(loaded_data, file)

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [ ]:
from stable_baselines3.common.monitor import Monitor

In [ ]:
eval_env = Monitor(env)

In [ ]:
mean_reward, std_reward = evaluate_policy(model, eval_env, n_eval_episodes=10, deterministic=True)
print(f"mean_reward={mean_reward:.2f} +/- {std_reward}")

mean_reward=-1949.70 +/- 721.9221633943648


In [ ]:
mean_reward, std_reward = evaluate_policy(model, eval_env, n_eval_episodes=10, deterministic=True)
print(f"mean_reward={mean_reward:.2f} +/- {std_reward}")

mean_reward=-1.20 +/- 0.4


In [ ]:
mean_reward, std_reward = evaluate_policy(model, env, n_eval_episodes=1, deterministic=True)
print(f"mean_reward={mean_reward:.2f} +/- {std_reward}")

/usr/local/lib/python3.10/dist-packages/stable_baselines3/common/evaluation.py:67: UserWarning: Evaluation environment is not wrapped with a ``Monitor`` wrapper. This may result in reporting modified episode lengths and rewards, if other wrappers happen to modify these. Consider wrapping environment first with ``Monitor`` wrapper.
  warnings.warn(


mean_reward=-2500.00 +/- 0.0


In [ ]:
# import gymnasium as gym
# from gymnasium import spaces
# import numpy as np

# class NQueensEnv(gym.Env):
#     def __init__(self, n=5):
#         super(NQueensEnv, self).__init__()
#         self.n = n  # Size of the board
#         self.action_space = spaces.Discrete(n * n)  # Action space
#         self.observation_space = spaces.MultiDiscrete([n] * n)  # Observation space
#         self.reset()

#     def reset(self, **kwargs):
#         #self.state = np.array([0 for i in range(self.n)])
#         self.state = np.random.choice(self.n, size=self.n, replace=False)  # Random initial state
#         return self.state, {}

#     def step(self, action):
#         row = action % self.n
#         col = action // self.n
#         self.state[row] = col
#         reward = self.calculate_reward()
#         done = reward == 0
#         return self.state, reward, done, False, {}

#     def calculate_reward(self):
#         # Initialize violations count
#         violations = 0

#         # Initialize sets to keep track of conflicts
#         col_set = set()
#         diag1_set = set()
#         diag2_set = set()

#         # Iterate through each queen
#         for row, col in enumerate(self.state):
#             # Calculate positions on diagonals
#             diag1 = col + row
#             diag2 = col - row

#             # Check for conflicts
#             if col in col_set or diag1 in diag1_set or diag2 in diag2_set:
#                 violations += 1

#             # Update sets
#             col_set.add(col)
#             diag1_set.add(diag1)
#             diag2_set.add(diag2)

#         # Reward is negative number of violations
#         return -violations

#     def is_terminal_state(self):
#         return self.calculate_reward() == 0  # Terminal state reached when there are no violations

#     def render(self, mode='human'):
#         board = [['_' for _ in range(self.n)] for _ in range(self.n)]
#         for i, row in enumerate(board):
#             row[self.state[i]] = 'Q'
#             print(' '.join(row))

# # Example of creating and using the environment
# if __name__ == "__main__":
#     env = NQueensEnv()
#     state, _ = env.reset()
#     print("Initial state:")
#     env.render()

#     # Example of taking a step
#     action = env.action_space.sample()
#     state, reward, done, truncated, info = env.step(action)
#     print("\nState after one step:")
#     env.render()
#     print(f"Reward: {reward}, Done: {done}")


Initial state:
Q _ _ _ _
_ _ Q _ _
_ _ _ _ Q
_ Q _ _ _
_ _ _ Q _

State after one step:
Q _ _ _ _
_ _ Q _ _
_ _ _ Q _
_ Q _ _ _
_ _ _ Q _
Reward: -2, Done: False


In [ ]:
episodes = 1

for ep in range(episodes):
    obs, info = env.reset()

    print(obs, "obs....")
    done = False
    print("episode ", ep)
    print("initial obs: ", obs)
    i = 0

    while not done or not truncated:
        i += 1

        action, _states = model.predict(obs)
        print(env.step(action))
        obs, rewards, done, truncated, info = env.step(action)

        print("action:", action, "new_state:", obs, "reward:", rewards, "done:", done, "truncated:" ,truncated)

        if done or truncated:
            break

env.close()

[5 2 4 6 3 7 0 1] obs....
episode  0
initial obs:  [5 2 4 6 3 7 0 1]
(array([5, 2, 4, 1, 3, 7, 0, 1]), -3, False, False, {})
action: 11 new_state: [5 2 4 1 3 7 0 1] reward: -3 done: False truncated: False
(array([5, 2, 7, 1, 3, 7, 0, 1]), -3, False, False, {})
action: 58 new_state: [5 2 7 1 3 7 0 1] reward: -3 done: False truncated: False
(array([5, 0, 7, 1, 3, 7, 0, 1]), -5, False, False, {})
action: 1 new_state: [5 0 7 1 3 7 0 1] reward: -5 done: False truncated: False
(array([5, 0, 7, 1, 3, 7, 2, 1]), -4, False, False, {})
action: 22 new_state: [5 0 7 1 3 7 2 1] reward: -4 done: False truncated: False
(array([3, 0, 7, 1, 3, 7, 2, 1]), -3, False, False, {})
action: 24 new_state: [3 0 7 1 3 7 2 1] reward: -3 done: False truncated: False
(array([3, 0, 7, 1, 3, 7, 2, 4]), -2, False, False, {})
action: 39 new_state: [3 0 7 1 3 7 2 4] reward: -2 done: False truncated: False
(array([3, 0, 7, 1, 3, 7, 2, 4]), -2, False, False, {})
action: 11 new_state: [3 0 7 1 3 7 2 4] reward: -2 done: Fal